Assignment 2

1. CIFAR-10 MLP

In [39]:
# PyTorch 라이브러리 import  *** 해당 cell을 수정하지 말 것 ***
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [40]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 랜덤으로 수평 뒤집기
    transforms.RandomCrop(32, padding=4),  # 랜덤 크롭 (패딩 추가)
    transforms.ToTensor(),  # 텐서로 변환
])

In [41]:
# TODO: CIFAR-10 training set 불러오기
train_dataset = datasets.CIFAR10(root='.', train=True, download=True, transform=transform)
batchsize = 64

valid_dataset = torch.utils.data.Subset(train_dataset, range(49000, 50000))
train_dataset = torch.utils.data.Subset(train_dataset, range(49000))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batchsize, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batchsize, shuffle=False)

Files already downloaded and verified


In [42]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    torch.manual_seed(2024) # 결과 재현을 위한 seed number 고정 *** 해당 line을 수정하지 말 것 ***
    # TODO: MLP 구성 layer들 선언 (Hint : Model input이 RGB 3채널 32x32 픽셀 이미지이므로 MLP 입력층 노드 개수는 32*32*3 = 3,072개임)
    self.mlp = nn.Linear(32*32*3, 4096)
    self.affine1 = nn.Linear(4096, 2048)
    self.batchnorm1 = nn.BatchNorm1d(2048)
    self.dropout1 = nn.Dropout(0.3)

    self.affine2 = nn.Linear(2048, 1024)
    self.batchnorm2 = nn.BatchNorm1d(1024)
    self.dropout2 = nn.Dropout(0.3)

    self.affine3 = nn.Linear(1024, 1024)
    self.batchnorm3 = nn.BatchNorm1d(1024)
    self.dropout3 = nn.Dropout(0.3)

    self.affine4 = nn.Linear(1024, 1024)
    self.batchnorm4 = nn.BatchNorm1d(1024)
    self.dropout4 = nn.Dropout(0.3)

    self.affine5 = nn.Linear(1024, 512)
    self.batchnorm5 = nn.BatchNorm1d(512)
    self.dropout5 = nn.Dropout(0.3)

    self.affine6 = nn.Linear(512, 256)
    self.batchnorm6 = nn.BatchNorm1d(256)

    self.affine7 = nn.Linear(256, 10)

    self.activation = nn.ReLU()

  def forward(self, x):
    x = x.view(-1, 32*32*3) # MLP가 이미지를 처리할 수 있도록 3차원 -> 1차원 벡터로 Flatten
    # TODO: forward pass 정의
    x = self.mlp(x)
    x = self.affine1(x)
    x = self.batchnorm1(x)
    x = self.activation(x)
    x = self.dropout1(x)

    x = self.affine2(x)
    x = self.batchnorm2(x)
    x = self.activation(x)
    x = self.dropout2(x)

    x = self.affine3(x)
    x = self.batchnorm3(x)
    x = self.activation(x)
    x = self.dropout3(x)

    x = self.affine4(x)
    x = self.batchnorm4(x)
    x = self.activation(x)
    x = self.dropout4(x)

    x = self.affine5(x)
    x = self.batchnorm5(x)
    x = self.activation(x)
    x = self.dropout5(x)

    x = self.affine6(x)
    x = self.batchnorm6(x)
    x = self.activation(x)

    x = self.affine7(x)
    return x

model = MLP().cuda()

In [43]:
# TODO: model 명의로 생성된 MLP 모델에 대해 학습 수행하기
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

num_epochs = 100
iter = 0
best_validation_acc = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for image, label in train_loader:
        image = image.cuda()
        label = label.cuda()
        output = model(image)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        iter += 1

        total_loss += loss.item()
    total_loss /= len(train_loader)
    print(f'[Epoch {epoch}]: Train Loss: {total_loss:.4f}', end=" ")

    model.eval()
    num_correct = 0
    with torch.no_grad():
        for image, label in valid_loader:
            image = image.cuda()
            label = label.cuda()
            output = model(image)
            pred = output.argmax(dim=1)
            num_correct += (pred == label).sum()

    validation_accuracy = num_correct / len(valid_dataset) * 100
    print(f"Valid Accuracy: {validation_accuracy:.4f}")
    if validation_accuracy > best_validation_acc:
        best_validation_acc = validation_accuracy
    elif validation_accuracy < best_validation_acc - 5:
        print(f'Early stopped at epoch {epoch + 1}')
        break

print(f'Total iterations: {iter}')

[Epoch 0]: Train Loss: 1.9525 Valid Accuracy: 34.4000
[Epoch 1]: Train Loss: 1.8061 Valid Accuracy: 36.9000
[Epoch 2]: Train Loss: 1.7474 Valid Accuracy: 38.2000
[Epoch 3]: Train Loss: 1.7013 Valid Accuracy: 38.7000
[Epoch 4]: Train Loss: 1.6667 Valid Accuracy: 42.4000
[Epoch 5]: Train Loss: 1.6382 Valid Accuracy: 44.4000
[Epoch 6]: Train Loss: 1.6116 Valid Accuracy: 43.4000
[Epoch 7]: Train Loss: 1.5840 Valid Accuracy: 46.4000
[Epoch 8]: Train Loss: 1.5631 Valid Accuracy: 46.2000
[Epoch 9]: Train Loss: 1.5438 Valid Accuracy: 44.2000
[Epoch 10]: Train Loss: 1.5292 Valid Accuracy: 49.2000
[Epoch 11]: Train Loss: 1.5074 Valid Accuracy: 46.8000
[Epoch 12]: Train Loss: 1.4963 Valid Accuracy: 49.2000
[Epoch 13]: Train Loss: 1.4854 Valid Accuracy: 51.6000
[Epoch 14]: Train Loss: 1.4773 Valid Accuracy: 50.1000
[Epoch 15]: Train Loss: 1.4618 Valid Accuracy: 48.8000
[Epoch 16]: Train Loss: 1.4527 Valid Accuracy: 51.0000
[Epoch 17]: Train Loss: 1.4399 Valid Accuracy: 53.8000
[Epoch 18]: Train Lo

In [45]:
# 학습된 모델 평가  *** 해당 cell을 수정하지 말 것 ***


test_dataset = datasets.CIFAR10(root='.', train=False, download=True,
                                 transform=transforms.ToTensor())
batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image = image.cuda()
    label = label.cuda()
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')

Files already downloaded and verified
Accuracy : 60.29 %


In [46]:
num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in train_loader:
    image = image.cuda()
    label = label.cuda()
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(train_dataset) * 100:.2f} %')

Accuracy : 63.52 %
